In [26]:
from crim_intervals import *
import pandas as pd
import ast
import matplotlib
from itertools import tee, combinations
import numpy as np

In [27]:
# THIS IS DEV COPY for use with CLOSE/EXACT Matches

import numpy as np

# Converts lists to tuples

def lists_to_tuples_a(el):
    if isinstance(el, list):
        return tuple(el)
    else:
        return el

# Filters for the length of the Presentation Type in the Classifier

def limit_offset_size(array, limit):
    under_limit = np.cumsum(array) <= limit
    return array[: sum(under_limit)]


# Gets the the list of offset differences for each group 

def get_offset_difference_list_a(group):
    # if we do sort values as part of the func call, then we don't need this first line
    group = group.sort_values("start_offset")
    group["next_offset"] = group.start_offset.shift(-1)
    offset_difference_list = (group.next_offset - group.start_offset).dropna().tolist()
    return offset_difference_list

# The classifications are done here

def classify_offsets_a(offset_difference_list):
    """
    Put logic for classifying an offset list here
    """
    #
    offset_difference_list = limit_offset_size(offset_difference_list, 5000)


    
    alt_list = offset_difference_list[::2]
    
    
    if len(set(offset_difference_list)) == 1 and len(offset_difference_list) > 1:
        return ("PEN", offset_difference_list)
    elif (len(offset_difference_list) %2 != 0) and (len(set(alt_list)) == 1):
        return ("ID", offset_difference_list)
    elif len(offset_difference_list) >= 1:
        return ("Fuga", offset_difference_list)
    else: 
        return ("Singleton", offset_difference_list)

   
# adds predicted type, offsets and entry numbers to the results

def predict_type_a(group):
    offset_differences = get_offset_difference_list_a(group)
    predicted_type, offsets = classify_offsets_a(offset_differences)

    group["predicted_type"] = [predicted_type for i in range(len(group))]
    group["offset_diffs"] = [offsets for i in range(len(group))]
    group["entry_number"] = [i + 1 for i in range(len(group))]

    return group

In [28]:
df = pd.read_csv(('ave_test_set.csv'), index_col=0)

df = df.drop(columns=["ema", "ema_url", "end_measure", "end_beat"])



In [38]:
classified = df.applymap(lists_to_tuples_a).sort_values("start_offset").groupby("pattern_matched").apply(predict_type_a)
classified["group_number"] = classified.groupby('pattern_generating_match').ngroup()
classified = classified[["group_number", "pattern_generating_match", "pattern_matched", "part", "start_measure", "start_beat", "entry_number", "start_offset", "sum_durs", "offset_diffs", "predicted_type"]]

# remove singleton entries

classified.drop(classified[classified['predicted_type'] == "Singleton"].index, inplace = True)

# now classified results, in order by OFFSET and ENTRY NUMBER, but the group numbers can overlap with each other.
classified.head(25)

classified = classified.sort_values(["group_number", "entry_number"])
classified= classified[["group_number", "entry_number", "pattern_matched", "part", "start_measure", "start_beat", "start_offset", "sum_durs", "offset_diffs", "predicted_type"]]
classified


,group_number,entry_number,pattern_matched,part,start_measure,start_beat,start_offset,sum_durs,offset_diffs,predicted_type
18,0,1,"[-2, -2, -2, -2, -2]",[Superius],19,1.0,144.0,14.0,"[14.0, 58.0, 456.0]",Fuga
19,0,2,"[-2, -2, -2, -2, -2]",[Superius],20,4.0,158.0,18.0,"[14.0, 58.0, 456.0]",Fuga
21,0,3,"[-2, -2, -2, -2, -2]",Tenor,28,1.0,216.0,10.0,"[14.0, 58.0, 456.0]",Fuga
20,0,4,"[-2, -2, -2, -2, -2]",[Superius],85,1.0,672.0,20.0,"[14.0, 58.0, 456.0]",Fuga
73,1,1,"[-2, -2, -2, -2, -3]",Tenor,42,1.0,328.0,22.0,"[0.0, 258.0, 302.0]",Fuga
...,...,...,...,...,...,...,...,...,...,...
0,40,1,"[4, 1, 2, 2, -3]",[Superius],1,1.0,0.0,32.0,"[16.0, 16.0, 16.0, 836.0]",Fuga
2,40,2,"[4, 1, 2, 2, -3]",Altus,3,1.0,16.0,32.0,"[16.0, 16.0, 16.0, 836.0]",Fuga
3,40,3,"[4, 1, 2, 2, -3]",Tenor,5,1.0,32.0,32.0,"[16.0, 16.0, 16.0, 836.0]",Fuga
4,40,4,"[4, 1, 2, 2, -3]",Bassus,7,1.0,48.0,32.0,"[16.0, 16.0, 16.0, 836.0]",Fuga


In [73]:

# r = classify["prev_ent_offset"] = classify["start_offset"]
classify = df.sort_values("start_offset").groupby("pattern_matched")
for pattern_matched, group in classify: 
    classify["prev_ent_offset"] = classify["start_offset"]
    print(prev_ent_offset) 
    # print(group) 

TypeError: 'DataFrameGroupBy' object does not support item assignment

In [79]:
# here we add the offset of the _previous_entry_ for any item in a group

def add_prev_entry(df):
    df["prev_entry_off"] = df["start_offset"].shift(1)
    return df

In [103]:
# now process the original match data df by:
# sorting by start offset, then group by pattern generating match and applying the function above for prev entry
# also give each group a number
# sort by group number and start offset


df2 = df.sort_values("start_offset").groupby('pattern_generating_match').apply(add_prev_entry)
df2["group_number"] = df2.groupby('pattern_generating_match').ngroup()
df2 = df2.sort_values(['group_number', 'start_offset'])
df2.head()

,pattern_generating_match,pattern_matched,piece_title,part,start_measure,start_beat,start_offset,end_offset,note_durations,sum_durs,prev_entry_off,group_number
18,"(-2, -2, -2, -2, -2)","[-2, -2, -2, -2, -2]",Ave Maria,[Superius],19,1.0,144.0,156.0,"[3.0, 1.0, 2.0, 2.0, 4.0, 2.0]",14.0,NaN,0
19,"(-2, -2, -2, -2, -2)","[-2, -2, -2, -2, -2]",Ave Maria,[Superius],20,4.0,158.0,172.0,"[3.0, 1.0, 4.0, 2.0, 4.0, 4.0]",18.0,144.0,0
21,"(-2, -2, -2, -2, -2)","[-2, -2, -2, -2, -2]",Ave Maria,Tenor,28,1.0,216.0,225.0,"[3.0, 1.0, 1.0, 1.0, 3.0, 1.0]",10.0,158.0,0
20,"(-2, -2, -2, -2, -2)","[-2, -2, -2, -2, -2]",Ave Maria,[Superius],85,1.0,672.0,688.0,"[4.0, 2.0, 4.0, 2.0, 4.0, 4.0]",20.0,216.0,0
73,"(-2, -2, -2, -2, -3)","[-2, -2, -2, -2, -3]",Ave Maria,Tenor,42,1.0,328.0,348.0,"[6.0, 2.0, 4.0, 4.0, 4.0, 2.0]",22.0,NaN,1


In [118]:
# now a boolean table for those entries that are 'same offset; as previous in a given group
df2["parallel_entry"] = df2["start_offset"] == df2["prev_entry_off"]
# return df2
df2.head(20)

,pattern_generating_match,pattern_matched,piece_title,part,start_measure,start_beat,start_offset,end_offset,note_durations,sum_durs,prev_entry_off,group_number,parallel_entry
18,"(-2, -2, -2, -2, -2)","[-2, -2, -2, -2, -2]",Ave Maria,[Superius],19,1.0,144.0,156.0,"[3.0, 1.0, 2.0, 2.0, 4.0, 2.0]",14.0,NaN,0,False
19,"(-2, -2, -2, -2, -2)","[-2, -2, -2, -2, -2]",Ave Maria,[Superius],20,4.0,158.0,172.0,"[3.0, 1.0, 4.0, 2.0, 4.0, 4.0]",18.0,144.0,0,False
21,"(-2, -2, -2, -2, -2)","[-2, -2, -2, -2, -2]",Ave Maria,Tenor,28,1.0,216.0,225.0,"[3.0, 1.0, 1.0, 1.0, 3.0, 1.0]",10.0,158.0,0,False
20,"(-2, -2, -2, -2, -2)","[-2, -2, -2, -2, -2]",Ave Maria,[Superius],85,1.0,672.0,688.0,"[4.0, 2.0, 4.0, 2.0, 4.0, 4.0]",20.0,216.0,0,False
73,"(-2, -2, -2, -2, -3)","[-2, -2, -2, -2, -3]",Ave Maria,Tenor,42,1.0,328.0,348.0,"[6.0, 2.0, 4.0, 4.0, 4.0, 2.0]",22.0,NaN,1,False
71,"(-2, -2, -2, -2, -3)","[-2, -2, -2, -2, -3]",Ave Maria,[Superius],42,1.0,328.0,348.0,"[6.0, 2.0, 4.0, 4.0, 4.0, 4.0]",24.0,328.0,1,True
74,"(-2, -2, -2, -2, -3)","[-2, -2, -2, -2, -3]",Ave Maria,Tenor,74,2.0,586.0,594.0,"[3.0, 1.0, 1.0, 1.0, 2.0, 2.0]",10.0,328.0,1,False
72,"(-2, -2, -2, -2, -3)","[-2, -2, -2, -2, -3]",Ave Maria,Altus,106,1.0,888.0,904.0,"[6.0, 2.0, 2.0, 2.0, 4.0, 4.0]",20.0,586.0,1,False
177,"(-2, -2, -2, -2, 1)","[-2, -2, -2, -2, 1]",Ave Maria,Altus,95,1.0,756.0,776.0,"[6.0, 2.0, 2.0, 2.0, 8.0, 4.0]",24.0,NaN,2,False
178,"(-2, -2, -2, -2, 1)","[-2, -2, -2, -2, 1]",Ave Maria,Altus,99,1.0,804.0,824.0,"[6.0, 2.0, 2.0, 2.0, 8.0, 4.0]",24.0,756.0,2,False


In [117]:
#filter out all the parallel entry items
df3 = df2[df2['parallel_entry'] != True] 
df3




,pattern_generating_match,pattern_matched,piece_title,part,start_measure,start_beat,start_offset,end_offset,note_durations,sum_durs,prev_entry_off,group_number,parallel_entry
18,"(-2, -2, -2, -2, -2)","[-2, -2, -2, -2, -2]",Ave Maria,[Superius],19,1.0,144.0,156.0,"[3.0, 1.0, 2.0, 2.0, 4.0, 2.0]",14.0,NaN,0,False
19,"(-2, -2, -2, -2, -2)","[-2, -2, -2, -2, -2]",Ave Maria,[Superius],20,4.0,158.0,172.0,"[3.0, 1.0, 4.0, 2.0, 4.0, 4.0]",18.0,144.0,0,False
21,"(-2, -2, -2, -2, -2)","[-2, -2, -2, -2, -2]",Ave Maria,Tenor,28,1.0,216.0,225.0,"[3.0, 1.0, 1.0, 1.0, 3.0, 1.0]",10.0,158.0,0,False
20,"(-2, -2, -2, -2, -2)","[-2, -2, -2, -2, -2]",Ave Maria,[Superius],85,1.0,672.0,688.0,"[4.0, 2.0, 4.0, 2.0, 4.0, 4.0]",20.0,216.0,0,False
73,"(-2, -2, -2, -2, -3)","[-2, -2, -2, -2, -3]",Ave Maria,Tenor,42,1.0,328.0,348.0,"[6.0, 2.0, 4.0, 4.0, 4.0, 2.0]",22.0,NaN,1,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,"(4, 1, 2, 2, -3)","[4, 1, 2, 2, -3]",Ave Maria,[Superius],1,1.0,0.0,24.0,"[4.0, 8.0, 4.0, 4.0, 4.0, 8.0]",32.0,NaN,40,False
2,"(4, 1, 2, 2, -3)","[4, 1, 2, 2, -3]",Ave Maria,Altus,3,1.0,16.0,40.0,"[4.0, 8.0, 4.0, 4.0, 4.0, 8.0]",32.0,0.0,40,False
3,"(4, 1, 2, 2, -3)","[4, 1, 2, 2, -3]",Ave Maria,Tenor,5,1.0,32.0,56.0,"[4.0, 8.0, 4.0, 4.0, 4.0, 8.0]",32.0,16.0,40,False
4,"(4, 1, 2, 2, -3)","[4, 1, 2, 2, -3]",Ave Maria,Bassus,7,1.0,48.0,72.0,"[4.0, 8.0, 4.0, 4.0, 4.0, 8.0]",32.0,32.0,40,False


In [122]:
# now run the classifer, but on DF from which the Parallel Entries have been removed!

classified2 = df3.applymap(lists_to_tuples_a).sort_values("start_offset").groupby("pattern_matched").apply(predict_type_a)
classified2["group_number"] = classified2.groupby('pattern_generating_match').ngroup()
classified2 = classified2[["group_number", "pattern_generating_match", "pattern_matched", "part", "start_measure", "start_beat", "entry_number", "start_offset", "sum_durs", "offset_diffs", "predicted_type"]]

# remove singleton entries

classified2.drop(classified[classified['predicted_type'] == "Singleton"].index, inplace = True)

# now classified results, in order by OFFSET and ENTRY NUMBER, but the group numbers can overlap with each other.


classified2 = classified2.sort_values(["group_number", "entry_number"])
classified2= classified2[["group_number", "entry_number", "pattern_matched", "part", "start_measure", "start_beat", "start_offset", "sum_durs", "offset_diffs", "predicted_type"]]
classified2

classified2.head(25)

,group_number,entry_number,pattern_matched,part,start_measure,start_beat,start_offset,sum_durs,offset_diffs,predicted_type
18,0,1,"[-2, -2, -2, -2, -2]",[Superius],19,1.0,144.0,14.0,"[14.0, 58.0, 456.0]",Fuga
19,0,2,"[-2, -2, -2, -2, -2]",[Superius],20,4.0,158.0,18.0,"[14.0, 58.0, 456.0]",Fuga
21,0,3,"[-2, -2, -2, -2, -2]",Tenor,28,1.0,216.0,10.0,"[14.0, 58.0, 456.0]",Fuga
20,0,4,"[-2, -2, -2, -2, -2]",[Superius],85,1.0,672.0,20.0,"[14.0, 58.0, 456.0]",Fuga
73,1,1,"[-2, -2, -2, -2, -3]",Tenor,42,1.0,328.0,22.0,"[258.0, 302.0]",Fuga
74,1,2,"[-2, -2, -2, -2, -3]",Tenor,74,2.0,586.0,10.0,"[258.0, 302.0]",Fuga
72,1,3,"[-2, -2, -2, -2, -3]",Altus,106,1.0,888.0,20.0,"[258.0, 302.0]",Fuga
177,2,1,"[-2, -2, -2, -2, 1]",Altus,95,1.0,756.0,24.0,"[48.0, 60.0, 8.0]",Fuga
178,2,2,"[-2, -2, -2, -2, 1]",Altus,99,1.0,804.0,24.0,"[48.0, 60.0, 8.0]",Fuga
176,2,3,"[-2, -2, -2, -2, 1]",[Superius],104,1.0,864.0,24.0,"[48.0, 60.0, 8.0]",Fuga
